Imports:

In [ ]:
############# IMPORTS ###############
from brian2 import *
from network import *
from input import *
from run import *
from analysis import *
from PIL import Image
import orjson
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys
import os
import pickle
from datetime import datetime

#BETA is 6 MS-1


Inputs:

In [2]:

##########GET INPUTS ############
def gen_brian_inputs():
    """
    Generates 3D Poisson input rates using Gabor filters for a set of images.
    This function performs the following steps:
    1. Defines parameters for Gabor filters including wavelengths, scaling factors, orientations, phase offsets, and aspect ratios.
    2. Creates Gabor filters using the specified parameters.
    3. Generates permutations of image identifiers.
    4. Loads and processes images from the specified path.
    5. Converts images to greyscale and normalizes them.
    6. Generates 3D Poisson input rates from the processed images using the Gabor filters.
    Returns:
        np.ndarray: A 3D array of Poisson input rates generated from the images.
    """

    import numpy as np

    lambdas = [0.8]  # Wavelengths
    betas = [1]  # Scaling factor for bandwidth
    thetas = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Orientations
    psis = [0, np.pi]  # Phase offsets
    gammas = [0.5]  # Aspect ratio
    size = 6  # Gabor filter sizes
    gabor_filters = GaborFilters(size, lambdas, betas, thetas, psis, gammas)

    from itertools import product

    permutations = list(product(["c", "v"], repeat=3))
    image_path = "data/3N2P/"
    # Create an np array with 8 images and each size 124x124:
    images = np.zeros((8, 128, 128))
    image_no = 0
    print("analysing data")
    for t, l, r in permutations:
        print(f"t: {t}, l: {l}, r: {r}")
        image_paths = image_path + f"t{t}_l{l}_r{r}.jpg"
        print(image_paths)
        image = Image.open(image_paths)
        greyscale = image.convert("L")
        greyscale = np.array(greyscale) / 255.0
        images[image_no] = greyscale
        image_no += 1

    _3d_poisson_inputs = generate_3d_poisson_rates_from_filters(
        images,
        gabor_filters,
        neuron_size=64,
        image_size=128,
    )
    # This has the shape num_images, neuron_size, neuron_size, num_filters

    return _3d_poisson_inputs



In [3]:
########## MAIN LOGIC ############
# Create equations and network
equations_container = EquationsContainer()
network = Network()

# Define Constants
# import file from weights/initial_setup.json as DATA
with open("weights/initial_setup.json", "rb") as file:
    DATA = orjson.loads(file.read())

N_LAYERS = 4  # Number of layers to create
NO_EPOCHS = 60
STIMULUS_LENGTH = 100 * ms
NUM_INPUTS = 8
STORAGE = None  # It worked but should be falsy so be concerned - I think I check whether it's None not False
DESCRIBE_NETWORK = False
input_lambda_e = 30 * nS
TEST_STIMULUS_LENGTH = 250 * ms
RESTORE = (False,)
STORE = (False,)
RADII = {
    "efe": {1: 8, 2: 12, 3: 16},
    "ele": {1: 2, 2: 2, 3: 2, 4: 2},
    "ebe": {2: 8, 3: 8, 4: 8},
    "eli": {1: 2, 2: 2, 3: 2, 4: 2},
    "ile": {1: 4, 2: 4, 3: 4, 4: 4},
}
AVG_NO_CONNECTIONS = {
    "efe": {0: 50, 1: 100, 2: 100, 3: 100},
    "ele": {1: 10, 2: 10, 3: 10, 4: 10},
    "ebe": {1: 10, 2: 10, 3: 10, 4: 10},
    "eli": {1: 10, 2: 10, 3: 10, 4: 10},
    "ile": {1: 30, 2: 30, 3: 30, 4: 30},
}

# Define neuron specifications:
exc_neuron_specs = NeuronSpecs(
    neuron_type="e",
    length=64,
    cm=500 * pF,
    g_leak=25 * nS,
    v_threshold=-53 * mV,
    v_reset=-57 * mV,
    v_rest=-74 * mV,
    v_reversal_e=0 * mV,
    v_reversal_i=-70 * mV,
    v_reversal_a=-90 * mV,
    sigma=0.015 * mV,
    t_refract=2 * ms,  # NEED TO ADD THIS
    tau_m=20 * ms,
    tau_ee=2 * ms,
    tau_ie=5 * ms,
    tau_a=80 * ms,
)

inh_neuron_specs = NeuronSpecs(
    neuron_type="i",
    length=32,
    cm=214 * pF,
    g_leak=18 * nS,
    v_threshold=-53 * mV,
    v_reset=-58 * mV,
    v_rest=-82 * mV,
    v_reversal_e=0 * mV,
    v_reversal_i=-70 * mV,
    sigma=0.015 * mV,
    t_refract=2 * ms,
    tau_m=12 * ms,
    tau_ei=2 * ms,
    tau_ii=5 * ms,
)

# Define synapse specifications:
efe_synapse_specs = SynapseSpecs(
    model=equations_container.synaptic_equations["stdp_model"],
    on_pre=equations_container.synaptic_equations["stdp_on_pre"],
    on_post=equations_container.synaptic_equations["stdp_on_post"],
    type="f",
    lambda_e=30 * nS,
    lambda_a=6 * nS,
    alpha_C=0.5,
    alpha_D=0.5,
    tau_c=5 * ms,
    tau_d=5 * ms,
    learning_rate=0.04, #NO IT ?aint
)

ele_synapse_specs = SynapseSpecs(
    model=equations_container.synaptic_equations["stdp_model"],
    on_pre=equations_container.synaptic_equations["stdp_on_pre"],
    on_post=equations_container.synaptic_equations["stdp_on_post"],
    type="l",
    lambda_e=20 * nS,
    lambda_a=6 * nS,
    alpha_C=0.5,
    alpha_D=0.5,
    tau_c=5 * ms,
    tau_d=5 * ms,
    learning_rate=0.04,
)
ebe_synapse_specs = SynapseSpecs(
    model=equations_container.synaptic_equations["stdp_model"],
    on_pre=equations_container.synaptic_equations["stdp_on_pre"],
    on_post=equations_container.synaptic_equations["stdp_on_post"],
    type="b",
    lambda_e=20 * nS,
    lambda_a=6 * nS,
    alpha_C=0.5,
    alpha_D=0.5,
    tau_c=5 * ms,
    tau_d=5 * ms,
    learning_rate=0.04,
)

eli_synapse_specs = SynapseSpecs(
    model=equations_container.synaptic_equations["inhib_non_stdp_model"],
    on_pre=equations_container.synaptic_equations["inhib_non_stdp_on_pre"],
    type="l",
    lambda_i=30 * nS,
)
ile_synapse_specs = SynapseSpecs(
    model=equations_container.synaptic_equations["excit_non_stdp_model"],
    on_pre=equations_container.synaptic_equations["excit_non_stdp_on_pre"],
    type="l",
    lambda_e=20 * nS,
    lambda_a=6 * nS,
)

_3d_poisson_rates = (
    gen_brian_inputs()
)  # This has the shape num_images, neuron_size, neuron_size, num_filters
# Create Neuron Groups:
create_neuron_groups(network, N_LAYERS, exc_neuron_specs, inh_neuron_specs)

# Create Synapses:
create_synapse_groups(
    network,
    N_LAYERS,
    RADII,
    AVG_NO_CONNECTIONS,
    exc_neuron_specs,
    inh_neuron_specs,
    efe_synapse_specs,
    ele_synapse_specs,
    ebe_synapse_specs,
    eli_synapse_specs,
    ile_synapse_specs,
    storage=STORAGE,
    data=DATA,
    store=STORE,
    restore=RESTORE,
)

####################    Sort inputs   ####################
print("Generating inputs")

# Got to make sure this is defined globally - can it be added to the network/included globally?
epoch_length = STIMULUS_LENGTH * NUM_INPUTS

# visualise_poisson_inputs(absolute_3d_poisson_rates)

network.store()

WARNING    /Users/jreid/Dropbox/dphil/programming/spikes/spikes/network/neurons.py:163: UserWarning: Mismatch in conductance parameters for type 'e'. Required: tau_ee, tau_ie, tau_a; Forbidden: tau_ei, tau_ii. Check values: tau_ee=2. ms, tau_ie=5. ms, tau_a=80. ms, tau_ei=None, tau_ii=None
  warnings.warn(
 [py.warnings]
WARNING    /Users/jreid/Dropbox/dphil/programming/spikes/spikes/network/neurons.py:151: UserWarning: Missing required parameters: v_reversal_a
  warnings.warn(f"Missing required parameters: {', '.join(missing_params)}")
 [py.warnings]
WARNING    /Users/jreid/Dropbox/dphil/programming/spikes/spikes/network/neurons.py:163: UserWarning: Mismatch in conductance parameters for type 'i'. Required: tau_ei, tau_ii; Forbidden: tau_ee, tau_ie, tau_a. Check values: tau_ei=2. ms, tau_ii=5. ms, tau_ee=None, tau_ie=None, tau_a=None
  warnings.warn(
 [py.warnings]


Parameter lambda_e is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter lambda_a is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter alpha_C is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter alpha_D is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter tau_c is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter tau_d is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter alpha_C is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile synapses
Parameter alpha_D is not provided. If l is a lateral synapse, it may not be needed if we are dealing with eli or ile syn

Convolving images: 100%|██████████| 8/8 [00:01<00:00,  5.47it/s]


creating 4 neuron layers
creating layer 1
adding layer1 neurons to network
adding layer1 neurons to network
creating layer 2
adding layer2 neurons to network
adding layer2 neurons to network
creating layer 3
adding layer3 neurons to network
adding layer3 neurons to network
creating layer 4
adding layer4 neurons to network
adding layer4 neurons to network
creating 4 synapse layers
creating synapses for layer 1
 *** creating efe synapses for layer 1 ***
synapse_name: efe_1
 *** connecting efe synapses for layer 1 ***
 *** RETRIEVING DATA FOR efe_1 ***


WARNING    Removing unsupported flag '-march=native' from compiler flags. [brian2.codegen.cpp_prefs]


Setting synapse parameters
set parameter: lambda_e as 30. nS
set parameter: lambda_a as 6. nS
set parameter: alpha_C as 0.5
set parameter: alpha_D as 0.5
set parameter: tau_c as 5. ms
set parameter: tau_d as 5. ms
set parameter: learning_rate as 0.04
Excluded values: ['lambda_i', 'A_plus']
 *** creating ele synapses for layer 1 *** 
synapse_name: ele_1
 *** RETRIEVING DATA FOR ele_1 ***
Setting synapse parameters
set parameter: lambda_e as 20. nS
set parameter: lambda_a as 6. nS
set parameter: alpha_C as 0.5
set parameter: alpha_D as 0.5
set parameter: tau_c as 5. ms
set parameter: tau_d as 5. ms
set parameter: learning_rate as 0.04
Excluded values: ['lambda_i', 'A_plus']
 *** creating eli synapses for layer 1 *** 
synapse_name: eli_1
 *** RETRIEVING DATA FOR eli_1 ***
Setting synapse parameters
set parameter: lambda_i as 30. nS
Excluded values: ['lambda_e', 'lambda_a', 'A_plus', 'alpha_C', 'alpha_D', 'tau_c', 'tau_d', 'learning_rate']
 *** creating ile synapses for layer 1 *** 
synaps

In [4]:
absolute_3d_poisson_rates = np.abs(_3d_poisson_rates)

train_input, test_input, poisson_neurons = wire_input_layer_brian(
    network,
    exc_neuron_specs,
    _3d_poisson_rates,
    beta=6000,
    radius=2,
    avg_no_neurons=AVG_NO_CONNECTIONS["efe"][0],
    epoch_length=epoch_length,
    stimulus_exposure_time=STIMULUS_LENGTH,
    stimulus_exposure_time_test=TEST_STIMULUS_LENGTH,
    input_lambda_e=input_lambda_e,
    storage=STORAGE,
    data=DATA,
    store=STORE,
)

namespace = {
    "input_lambda_e": input_lambda_e,
    "timed_input": train_input,
    "epoch_length": epoch_length,
}

########## WIRING INPUT LAYER ##########
num_neurons: 4096
beta:6000
[[ -3.42097644   3.42097644 -13.03014424 ...  11.47797395  -6.79757236
    6.79757236]
 [  5.45488266  -5.45488266  -9.39063459 ...   9.49826663  -4.85723573
    4.85723573]
 [-18.70248939  18.70248939 -14.41011463 ...  18.18076731 -15.29517811
   15.29517811]
 ...
 [-18.06289159  18.06289159 -14.04585755 ...  17.87713255 -14.55593652
   14.55593652]
 [ 17.17626952 -17.17626952   2.74656764 ...  14.20043406 -12.23666791
   12.23666791]
 [ 28.40308938 -28.40308938   1.00695973 ...   4.94283687  -0.6583227
    0.6583227 ]] Hz
beta:6000
[[ -3.42097644   3.42097644 -13.03014424 ...  11.47797395  -6.79757236
    6.79757236]
 [  5.45488266  -5.45488266  -9.39063459 ...   9.49826663  -4.85723573
    4.85723573]
 [-18.70248939  18.70248939 -14.41011463 ...  18.18076731 -15.29517811
   15.29517811]
 ...
 [-18.06289159  18.06289159 -14.04585755 ...  17.87713255 -14.55593652
   14.55593652]
 [ 17.17626952 -17.17626952   2.746567

In [5]:
initial_weights = {}
synapse_specs = [
    efe_synapse_specs,
    ele_synapse_specs,
    ebe_synapse_specs,
    eli_synapse_specs,
    ile_synapse_specs,
]
for synapse_spec in synapse_specs:
    synapse_type = (
        f"{synapse_spec.recent_a}{synapse_spec.type}{synapse_spec.recent_e}"
    )
    initial_weights[synapse_type] = {}
    for layer, synapse_group in synapse_spec.synapse_objects.items():
        initial_weights[synapse_type][layer] = {
            "i": np.array(synapse_group[0].i),
            "j": np.array(synapse_group[0].j),
            "w": np.array(synapse_group[0].w),
            "delay": np.array(synapse_group[0].delay),
        }

os.makedirs("results/full_monty", exist_ok=True)
with open("results/full_monty/initial_weights_ii.pkl", "wb") as file:
    pickle.dump(initial_weights, file)


In [6]:
print(equations_container.synaptic_equations["stdp_model"])


                    dapre/dt = -apre/tau_c : 1 (event-driven)
                    dapost/dt = -apost/tau_d : 1 (event-driven)
                    lambda_e: siemens
                    lambda_a: siemens
                    alpha_C: 1
                    alpha_D: 1
                    tau_c: second
                    tau_d: second
                    w: 1
                    plasticity: 1
                    learning_rate: 1
                    


In [7]:
synapses_pfe_post_0 = [obj for obj in network.objects if isinstance(obj, Synapses) and obj.name == "pfe_post_0"]
print(synapses_pfe_post_0)

[Synapses(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='pfe_post_0')]


In [8]:
# Is it a poisson at small time steps?

In [9]:
NO_EPOCHS = 60


In [10]:
# ####################    MONITOR NETWORK   ####################
# spike_monitor_0 = SpikeMonitor(poisson_neurons,record=True)
# spike_monitor_1 = SpikeMonitor(exc_neuron_specs.neuron_groups[1],record=True)
# spike_monitor_2 = SpikeMonitor(exc_neuron_specs.neuron_groups[2],record=True)
# spike_monitor_3 = SpikeMonitor(exc_neuron_specs.neuron_groups[3],record=True)
# spike_monitor_4 = SpikeMonitor(exc_neuron_specs.neuron_groups[4],record=True)
# spike_monitors = {0: spike_monitor_0, 1: spike_monitor_1, 2: spike_monitor_2, 3: spike_monitor_3, 4: spike_monitor_4}
# for key, spike_monitor in spike_monitors.items():
#     network.add(spike_monitor)

In [ ]:
# RUN THE NETWORK::
for obj in network.objects:
    if isinstance(obj, Synapses):
        if obj.name == "efe_2":
            print("found efe_2")
            efe_2 = obj
            print(efe_2.lambda_a)
print(type(efe_2))
timed_input = namespace["timed_input"]
for epoch in range(NO_EPOCHS):
    print(f"Running epoch no {epoch}")
    print(" -- Efe_2 state:")
    print(" -- weights:")
    print(" --" + efe_2.w[10000:10005])
    print(" -- plasticity:")
    print(" --" + efe_2.plasticity)
    print(" -- learning rate:")
    print(" --" + efe_2.learning_rate[10000:10005])
    print(" -- pre:")
    print(" --" + efe_2.apre[10000:10005])
    print(" -- post:")
    print(" --" + efe_2.apost[10000:10005])
    print(" --" + efe_2.ga[10000:10010])
    if epoch > 0:
        print("epoch>0")
        print(" --" + efe_2.w)
        print(current_list)
        changed_values = [
            x-y for x, y in zip(efe_2.w, current_list) if x != y
        ]
        print(
            f"changed_values (len={len(changed_values)})")
    current_list = np.array([])
    for index in efe_2.w:
        current_list = np.append(current_list, index)
    print(current_list[:25])
    print(f"running epoch no {epoch+1}")
    print(f"current time {network.t}")
    network.run(STIMULUS_LENGTH * 8)
9

synapse object found:
eli_4
synapse object found:
ile_2
synapse object found:
ele_4
synapse object found:
ile_1
synapse object found:
pfe_post_0
synapse object found:
ebe_3
synapse object found:
efe_2
found efe_2
<efe_2.lambda_a: array([6., 6., 6., ..., 6., 6., 6.]) * nsiemens>
synapse object found:
ele_3
synapse object found:
eli_3
synapse object found:
ele_1
synapse object found:
ele_2
synapse object found:
efe_3
synapse object found:
ile_3
synapse object found:
eli_2
synapse object found:
efe_1
synapse object found:
ile_4
synapse object found:
ebe_2
synapse object found:
ebe_4
synapse object found:
eli_1
<class 'brian2.synapses.synapses.Synapses'>
running epoch no 0
efe_2 state:
weights:
[0.35929346 0.52093827 0.12362797 0.64789266 0.77037459]
plasticity:
<efe_2.plasticity: array([1., 1., 1., ..., 1., 1., 1.])>
learning rate:
[0.04 0.04 0.04 0.04 0.04]
pre:
[0. 0. 0. 0. 0.]
post:
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] S
[0.0617023  0.58446658 0.39013265 0.68991404 0.399593

BrianObjectException: Error encountered with object named 'e_1_stateupdater'.
Object was created here (most recent call only, full details in debug log):
  File '/Users/jreid/Dropbox/dphil/programming/spikes/spikes/network/neurons.py', line 282, in create_neurons
    neurons = NeuronGroup(

An error occurred when preparing an object. (See above for original error message and traceback.)

In [ ]:
for obj in network.objects:
    if isinstance(obj, Synapses):
        print("synapse object found:")
        if obj.name == "efe_2":
            print("found efe_2")
            efe_2 = obj
            apres = obj.get_states(["apre"])['apre']
            print(len(apres[apres != 0]))
            print(len(apres))
            print(apres[10000:10025])

In [ ]:
for obj in network.objects:
    if isinstance(obj, NeuronGroup):
        print("Neuron object found:")
        print(obj.name)
        print(obj.get_states())
        print(obj.get_states(["ga"]))


In [ ]:

spike_monitor_0 = SpikeMonitor(poisson_neurons, record=True)
spike_monitor_1 = SpikeMonitor(exc_neuron_specs.neuron_groups[1], record=True)
spike_monitor_2 = SpikeMonitor(exc_neuron_specs.neuron_groups[2], record=True)
spike_monitor_3 = SpikeMonitor(exc_neuron_specs.neuron_groups[3], record=True)
spike_monitor_4 = SpikeMonitor(exc_neuron_specs.neuron_groups[4], record=True)
monitors = {
    0: spike_monitor_0,
    1: spike_monitor_1,
    2: spike_monitor_2,
    3: spike_monitor_3,
    4: spike_monitor_4,
}
for layer, monitor in monitors.items():
    network.add(monitor)
    print(f"added monitor for layer {layer}")
print("running testing epoch")


In [ ]:
timed_input = test_input
namespace["timed_input"] = test_input
epoch_length = TEST_STIMULUS_LENGTH * NUM_INPUTS
for obj in network.objects:
    if isinstance(obj, Synapses):
        try:
            Synapses.plasticity = 0
            print("synapse set to 0:")
            print(obj.name)
            print(obj.plasticity)
        except:
            print("no plasticity attribute")

In [ ]:
network.run(TEST_STIMULUS_LENGTH * 8)

In [ ]:
def get_all_relevant_data(
    Network,
    monitors,
    efe_synapse_specs,
    ele_synapse_specs,
    ebe_synapse_specs,
    eli_synapse_specs,
    ile_synapse_specs,
):
    """
    Get all relevant data from the network and store it in a dictionary.
    Returns:
        dict: A dictionary containing all relevant data from the network.
        weights (dict): A dictionary containing the weights of all synapses.
        spikes (dict): A dictionary containing the spike times of all excitatory neurons contained within the monitors object.
    """
    weights_and_delays = {}
    synapses = {
        "efe": efe_synapse_specs,
        "ele": ele_synapse_specs,
        "ebe": ebe_synapse_specs,
        "eli": eli_synapse_specs,
        "ile": ile_synapse_specs,
    }
    for synapse_type_str, synapse_type_object in synapses.items():
        dictionary = {}
        synapse_groups = synapse_type_object.synapse_objects
        layer = 0
        for key, item in synapse_groups.items():
            dictionary[layer] = {
                "i": np.array(item[0].i),
                "j": np.array(item[0].j),
                "w": np.array(item[0].w),
                "delay": np.array(item[0].delay)
            }
            layer += 1
        weights_and_delays[synapse_type_str] = dictionary
    spikes = {}
    for key, item in monitors.items():
        spikes[key] = dict(item.spike_trains())
    return {"weights_and_delays": weights_and_delays, "spikes": spikes}
data_in_network = get_all_relevant_data(
    network,
    monitors,
    efe_synapse_specs,
    ele_synapse_specs,
    ebe_synapse_specs,
    eli_synapse_specs,
    ile_synapse_specs,
)
print(type(data_in_network))
os.makedirs("results/full_monty", exist_ok=True)
with open("results/full_monty/dataset_ii.pkl", "wb") as file:
    pickle.dump(data_in_network, file)

for layer, monitor in monitors.items():
    print(f"number of spikes in layer {layer}: {len(monitor.i)}")


In [ ]:
for key, item in monitors.items():
    print(f"Layer {key} has {len(item.t)} spikes")
    plt.figure()
    plt.hist(item.t/ms, bins=int(max(item.t/ms)), cumulative=True, histtype='step', label=f"Layer {key}")
    plt.title(f"Cumulative Spikes for layer {key}")
    plt.xlabel("Time (ms)")
    plt.ylabel("Cumulative Spike Count")
    plt.legend()
# for i in range(len(voltage_monitor_1.v)):
#     plt.figure()
#     plt.plot(voltage_monitor_1.t/ms, voltage_monitor_1.v[i])
#     plt.title(f"Voltage of Neuron {i} over Time")
#     plt.xlabel("Time (ms)")
#     plt.ylabel("Voltage (V)")
#     plt.show()

# IF BETA DOESN"T WORK IT SUGGESTS THERE's SOMETHING WRONG WITH THE INPUTS, maybe the % is doing something weird or t,i got swapped around?

In [ ]:
weights = efe_2.w
from matplotlib import colors
plt.figure()
#weights is a 1D array of the weights of every synapse - make a histogram between 1 and 0
plt.hist(weights, bins=100, range=(0, 1))
plt.title("Histogram of Weights")
plt.xlabel("Weight")
plt.ylabel("Number of Synapses")
plt.show()
